In [1]:
!pip install pandas
!pip install numpy

In [15]:
from smpl.smpl_numpy import SMPL 

smpl_model = SMPL(sex='neutral', model_dir='assets/SMPL_NEUTRAL.pkl')
# v_sel = smpl_model.vertex_joint_selector 


In [16]:
    import pandas as pd
    import numpy as np
    # SMPL in canonical space
    big_pose_smpl_param = {}
    big_pose_smpl_param['R'] = np.eye(3).astype(np.float32)
    big_pose_smpl_param['Th'] = np.zeros((1,3)).astype(np.float32)
    big_pose_smpl_param['shapes'] = np.zeros((1,10)).astype(np.float32)
    big_pose_smpl_param['poses'] = np.zeros((1,72)).astype(np.float32)
    big_pose_smpl_param['poses'][0, 5] = 45/180*np.array(np.pi)
    big_pose_smpl_param['poses'][0, 8] = -45/180*np.array(np.pi)
    big_pose_smpl_param['poses'][0, 23] = -30/180*np.array(np.pi)
    big_pose_smpl_param['poses'][0, 26] = 30/180*np.array(np.pi)

    big_pose_xyz, x = smpl_model(big_pose_smpl_param['poses'], big_pose_smpl_param['shapes'].reshape(-1))
    big_pose_xyz = (np.matmul(big_pose_xyz, big_pose_smpl_param['R'].transpose()) + big_pose_smpl_param['Th']).astype(np.float32)

    # obtain the original bounds for point sampling
    big_pose_min_xyz = np.min(big_pose_xyz, axis=0)
    big_pose_max_xyz = np.max(big_pose_xyz, axis=0)
    big_pose_min_xyz -= 0.05
    big_pose_max_xyz += 0.05
    big_pose_world_bound = np.stack([big_pose_min_xyz, big_pose_max_xyz], axis=0)

In [20]:
Export OBJ 
# If `output` contains more than two values, we unpack them all:
vertices, joints = big_pose_xyz, x

# Convert the vertices to numpy array
vertices = vertices.squeeze()

# Save as OBJ file using trimesh
faces = smpl.faces
mesh = trimesh.Trimesh(vertices, faces)
mesh.export('smpl_model.obj')

'# https://github.com/mikedh/trimesh\nv 0.04351503 0.49372151 0.08987843\nv 0.03894845 0.48089752 0.10028563\nv 0.04970802 0.47556823 0.09171281\nv 0.05533651 0.48443544 0.08076887\nv 0.04633762 0.46776316 0.09950736\nv 0.03713237 0.46981719 0.10575732\nv 0.05684998 0.46796003 0.08280023\nv 0.06143441 0.47372186 0.07100473\nv 0.06235274 0.43164000 0.07826696\nv 0.06658909 0.42575219 0.06941936\nv 0.06559006 0.43801582 0.06822643\nv 0.06066589 0.44063032 0.07719857\nv 0.06247207 0.40514904 0.07963220\nv 0.05718689 0.39723283 0.08939893\nv 0.05960599 0.38648561 0.08129017\nv 0.06422605 0.39372674 0.07029901\nv 0.02499029 0.41499093 0.11002117\nv 0.03019291 0.40828639 0.11006422\nv 0.03419499 0.41486982 0.10799772\nv 0.02818034 0.42070934 0.10777250\nv 0.01566150 0.42557716 0.11422893\nv 0.01659585 0.43030322 0.11000043\nv 0.01177993 0.43498570 0.11434796\nv 0.01164227 0.42971283 0.11972290\nv 0.01954889 0.40966997 0.11106900\nv 0.02521848 0.40267429 0.11097448\nv 0.02037634 0.42093295 0.

In [4]:
obj = 'assets/SMPL_NEUTRAL.pkl'


obj_smpl = pd.read_pickle('assets/SMPL_NEUTRAL.pkl')
obj_smpl.get('J_regressor')[0][0]

<1x6890 sparse matrix of type '<class 'numpy.float64'>'
	with 27 stored elements in Compressed Sparse Column format>

In [5]:
# obj_smpl.get('J')
face= {
        'nose': 332,
        'reye': 6260,
        'leye': 2800,
        'rear':	4071,
        'lear':	583,
}
face.values()

dict_values([332, 6260, 2800, 4071, 583])

In [18]:
# obj_smpl.get('J')
face= {
        'nose': 332,
        'reye': 6260,
        'leye': 2800,
        'rear':	4071,
        'lear':	583,
}
# Extract the indices from the face dictionary
indices = np.array(list(face.values()))
print(indices)
# Use the indices to directly get the corresponding rows from big_pose_xyz
face_xyz = big_pose_xyz[indices]

# Print the result
print(face_xyz,indices[0:3],indices[3:5])

[ 332 6260 2800 4071  583]
[[ 0.00242026  0.41171256  0.1370538 ]
 [-0.03157485  0.44699487  0.09797596]
 [ 0.03461343  0.4475619   0.09933882]
 [-0.07265673  0.41884518  0.00539076]
 [ 0.07415988  0.42008573  0.00802347]] [ 332 6260 2800] [4071  583]


In [20]:
        ###############################**********************########################################## change 2 
        
        # obtain the original bounds for dense point sampling on the face
        xyz_face_min_xyz = np.min(face_xyz, axis=0)
        xyz_face_max_xyz = np.max(face_xyz, axis=0)
        xyz_face_min_xyz -= 0.05
        xyz_face_max_xyz += 0.05
        # xyz_face_world_bound = np.stack([xyz_face_min_xyz, xyz_face_max_xyz], axis=0)
        

        xyz2 = np.append(face_xyz,big_pose_xyz,axis=0)
        # xyz2 = np.append(xyz_hand,xyz2,axis=0)

        # Define the number of points along each axis (resolution of the grid)
        num_points = 10  # For example, 10 points along each axis
        
        # Create a grid of points within the bounding box
        x = np.linspace(xyz_face_min_xyz[0], xyz_face_max_xyz[0], num_points)
        y = np.linspace(xyz_face_min_xyz[1], xyz_face_max_xyz[1], num_points)
        z = np.linspace(xyz_face_min_xyz[2], xyz_face_max_xyz[2], num_points)

        # Generate all combinations of x, y, z
        X, Y, Z = np.meshgrid(x, y, z)
        # Reshape the grid to get a flat list of 3D points
        xyz2 = np.vstack([X.ravel(), Y.ravel(), Z.ravel()]).T
        print(xyz2.shape)
    
        ###############################**********************########################################## change 2 

(1000, 3)


In [21]:
print(xyz2
     )

print("done!")

[[-0.12265673  0.36171255 -0.04460924]
 [-0.12265673  0.36171255 -0.0188689 ]
 [-0.12265673  0.36171255  0.00687144]
 ...
 [ 0.12415989  0.4975619   0.13557312]
 [ 0.12415989  0.4975619   0.16131346]
 [ 0.12415989  0.4975619   0.1870538 ]]
done!


In [167]:
    # obtain the original bounds for point sampling
    face_xyz_max_xyz = np.max(face_xyz, axis=0)
    face_xyz_min_xyz = np.min(face_xyz, axis=0)
    face_xyz_min_xyz -= 0.05
    face_xyz_max_xyz += 0.05
    face_xyz_world_bound = np.stack([face_xyz_min_xyz, face_xyz_max_xyz], axis=0)

In [168]:
face_xyz_world_bound

array([[-0.12265673,  0.36171255, -0.04460924],
       [ 0.12415989,  0.4975619 ,  0.1870538 ]], dtype=float32)

In [197]:
xyz = big_pose_xyz
xyz2 = np.append(face_xyz,xyz,axis=0)
for x in range(1000):
    xyz2 = np.append(face_xyz,xyz2,axis=0)

In [198]:
print(xyz.shape ,xyz2.shape,face_xyz.shape)

(6890, 3) (11895, 3) (5, 3)


In [199]:
xyz2.shape[0]

11895